<a href="https://colab.research.google.com/github/Seong-JiHyeon/NLTK/blob/master/nltk_machinelearning(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 학습활동. [난이도 상] 지도학습 기반, G&I 문장 경계선 구분하기 위한 결정요소 파악하기

nltk 교재 6장의 2.1 Sentence Segmentation에 있는 예제를 학습하고, 다음 차례에 따라, G&I에 적용해 보자.
1. sentence_tokenized의 폴더(Genomics-Informatics-Corpus-master\GNI Corpus 1.0\sentence_tokenized)에 있는 파일을, 임의로 8:2 정도로 나누어, G&I train set와 G&I test set을 준비해 보자. 
2. G&I 코퍼스에 대하여, white space를 지우고, raw 버젼과 인덱싱 작업 등 약간의 사전 작업을 해야 할 것임 (문제 설명 동영상 참고할 것)
3. G&I test set에 대하여, 1) 교재의 brown 코퍼스를 기반하여, 트레이닝 시킨 모델(Naive Bayesian)과, 2) G&I train set에 기반하여 트레이닝 시킨 모델(Naive Bayesian)을 각각 적용하여, evaluation 결과를 비교해 보자.
4. 두개의 모델 각각에 대해서, show_most_informative() 함수를 사용해서 결과를 살펴 보고, 그 의미를 말로 설명해 볼 것.
- sentence_tokenized 원본 폴더의 데이터에도 에러가 있을 수 있으므로, 적어도 2개 이상의 GNI 논문에 대해서 문장분리 작업을 눈으로 확인하여 Test Set을 점검하고 evaluation 결과를 confirm 해 줄 것.

In [0]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My Drive/인공지능 수업 데이터 파일

/content/gdrive/My Drive/인공지능 수업 데이터 파일


In [0]:
def segment_sentences(words):
    start = 0
    sents = []
    for i, word in enumerate(words):
        if word in '.?!' and classifier.classify(punct_features(words, i)) == True:
            sents.append(words[start:i+1])
            start = i+1
    if start < len(words):
        sents.append(words[start:])
    return sents

In [0]:
# 수정된 sentence_tokenized에 대한 작업
corpus_root = "/content/gdrive/My Drive/인공지능 수업 데이터 파일/sentence_tokenized/"
gniCorpus = nltk.corpus.PlaintextCorpusReader(corpus_root, ".*\.txt", encoding='utf-8')

In [0]:
gniSents = gniCorpus.sents()
gniSents[:10]

[['Title',
  ':',
  'Survey',
  'of',
  'the',
  'Applications',
  'of',
  'NGS',
  'to',
  'Whole',
  '-',
  'Genome',
  'Sequencing',
  'and',
  'Expression',
  'Profiling'],
 ['Recently',
  ',',
  'the',
  'technologies',
  'of',
  'DNA',
  'sequence',
  'variation',
  'and',
  'gene',
  'expression',
  'profiling',
  'have',
  'been',
  'used',
  'widely',
  'as',
  'approaches',
  'in',
  'the',
  'expertise',
  'of',
  'genome',
  'biology',
  'and',
  'genetics',
  '.'],
 ['The',
  'application',
  'to',
  'genome',
  'study',
  'has',
  'been',
  'particularly',
  'developed',
  'with',
  'the',
  'introduction',
  'of',
  'the',
  'next',
  '-',
  'generation',
  'DNA',
  'sequencer',
  '(',
  'NGS',
  ')',
  'Roche',
  '/',
  '454',
  'and',
  'Illumina',
  '/',
  'Solexa',
  'systems',
  ',',
  'along',
  'with',
  'bioinformation',
  'analysis',
  'technologies',
  'of',
  'whole',
  '-',
  'genome',
  'de',
  'novo',
  'assembly',
  ',',
  'expression',
  'profiling',
  ',

In [0]:
# 수정 전 sentence_tokenized 폴더에 대한 작업
corpus_root = "/content/gdrive/My Drive/인공지능 수업 데이터 파일/Genomics-Informatics-Corpus-master/GNI Corpus 1.0/sentence_tokenized/"
gniCorpus = nltk.corpus.PlaintextCorpusReader(corpus_root, ".*\.txt", encoding="utf-8")

gniRaws = gniCorpus.raw().split('\n')
gniRaws = [nltk.word_tokenize(x) for x in gniRaws]
gniSents = [x for x in gniRaws if len(x) > 5] # 임의로 5개 정도 이하인 줄 단위 문장들은 버림

In [0]:
gniSents[:10]

[['Editor',
  '’',
  's',
  'Introduction',
  'to',
  'This',
  'Issue',
  '(',
  'G',
  '&',
  'I',
  '15:3',
  ',',
  '2017',
  ')',
  '.'],
 ['Genomics',
  '&',
  'Informatics',
  'Vol',
  '.',
  '15',
  ',',
  'No',
  '.',
  '3',
  ',',
  '2017eISSN2234-0742Genomics',
  'Inform',
  '2017',
  ';',
  '15',
  '(',
  '3',
  ')',
  ':81https',
  ':',
  '//doi.org/10.5808/GI.2017.15.3.81*Corresponding',
  'author',
  ':',
  'Tel',
  ':',
  '+82-2-2258-7343',
  ',',
  'Fax',
  ':',
  '+82-2-537-0572',
  ',',
  'E-mail',
  ':',
  'yejun',
  '@',
  'catholic.ac.kr'],
 ['Yeun-Jun',
  'Chung*Precision',
  'Medicine',
  'Research',
  'Center',
  ',',
  'College',
  'of',
  'Medicine',
  ',',
  'The',
  'Catholic',
  'University',
  'of',
  'Korea',
  ',',
  'Seoul',
  '06591',
  ',',
  'KoreaIn',
  'this',
  'issue',
  ',',
  'Drs',
  '.',
  'Sun',
  'Ho',
  'Lee',
  'and',
  'Wung',
  'Joo',
  'Song',
  'report',
  'copy',
  'number',
  'variations',
  'in',
  '42',
  'Korean',
  'patients',


In [0]:
len(gniSents) # raw 버젼과 sentence 단위 버젼

113090

In [0]:
tokens = []
boundaries = set()
offset = 0
for sent in gniSents:
        tokens.extend(sent)
        offset += len(sent)
        boundaries.add(offset-1)

In [0]:
def punct_features(tokens, i):
        return {'next-word-capitalized': tokens[i+1][0].isupper(),
                'prev-word': tokens[i-1].lower(),
                'punct': tokens[i],
                'prev-word-is-one-char': len(tokens[i-1]) == 1}

In [0]:
boundaries

{524291,
 655366,
 393223,
 262153,
 524301,
 786446,
 15,
 786454,
 393245,
 655390,
 524320,
 262180,
 786470,
 42,
 262191,
 655412,
 131125,
 393278,
 786494,
 393285,
 524358,
 131147,
 262223,
 262229,
 393301,
 786518,
 524380,
 131167,
 96,
 655457,
 393315,
 655475,
 131189,
 786552,
 127,
 524416,
 524421,
 262278,
 655497,
 393354,
 131217,
 786582,
 655513,
 262298,
 524450,
 131241,
 172,
 262316,
 393397,
 655541,
 786618,
 262332,
 131271,
 200,
 262354,
 786644,
 655573,
 393432,
 524515,
 231,
 524521,
 131310,
 786673,
 524531,
 262394,
 524540,
 393475,
 262,
 524552,
 655624,
 786697,
 131340,
 655640,
 393497,
 131354,
 262429,
 524576,
 294,
 786727,
 131370,
 393520,
 262449,
 131383,
 655672,
 393538,
 655683,
 131399,
 329,
 786767,
 262486,
 655703,
 393562,
 524636,
 352,
 262508,
 786806,
 376,
 655736,
 524669,
 393602,
 131464,
 262537,
 524697,
 655772,
 786857,
 429,
 131501,
 262573,
 393651,
 262585,
 442,
 786880,
 655815,
 131536,
 465,
 131542,
 474

In [0]:
featuresets = [(punct_features(tokens, i), (i in boundaries))
        for i in range(1, len(tokens)-1)
                if tokens[i] in '.?!']

In [0]:
featuresets[:10]

[({'next-word-capitalized': True,
   'prev-word': 'genetics',
   'prev-word-is-one-char': False,
   'punct': '.'},
  True),
 ({'next-word-capitalized': True,
   'prev-word': 'genotyping',
   'prev-word-is-one-char': False,
   'punct': '.'},
  True),
 ({'next-word-capitalized': True,
   'prev-word': 'data',
   'prev-word-is-one-char': False,
   'punct': '.'},
  True),
 ({'next-word-capitalized': True,
   'prev-word': 'assembly',
   'prev-word-is-one-char': False,
   'punct': '.'},
  True),
 ({'next-word-capitalized': True,
   'prev-word': 'sequencing',
   'prev-word-is-one-char': False,
   'punct': '.'},
  True),
 ({'next-word-capitalized': True,
   'prev-word': 'samples',
   'prev-word-is-one-char': False,
   'punct': '.'},
  True),
 ({'next-word-capitalized': True,
   'prev-word': 'species',
   'prev-word-is-one-char': False,
   'punct': '.'},
  True),
 ({'next-word-capitalized': True,
   'prev-word': 'sequences',
   'prev-word-is-one-char': False,
   'punct': '.'},
  True),
 ({'next-

In [0]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]

In [0]:
# treebank_raw 기반
treebank = nltk.corpus.treebank_raw.sents()

In [0]:
tokens = []
boundaries = set()
offset = 0
for sent in treebank:
        tokens.extend(sent)
        offset += len(sent)
        boundaries.add(offset-1)

In [0]:
featuresets = [(punct_features(tokens, i), (i in boundaries))
        for i in range(1, len(tokens)-1)
                if tokens[i] in '.?!']

In [0]:
train_set = featuresets

In [0]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)
print("Sentence Segmentation:", nltk.classify.accuracy(classifier, test_set))

Sentence Segmentation: 0.6270304226868886


In [0]:
classifier.show_most_informative_features() #gniCorpus 수정 후를 train set으로 설정했을 경우

Most Informative Features
               prev-word = '0'             False : True   =    139.7 : 1.0
               prev-word = 'e'             False : True   =    130.7 : 1.0
               prev-word = 'v'             False : True   =     83.0 : 1.0
               prev-word = 'f'             False : True   =     81.4 : 1.0
               prev-word = 'i'             False : True   =     77.4 : 1.0
               prev-word = 'genes'          True : False  =     72.5 : 1.0
               prev-word = 'dr'            False : True   =     69.0 : 1.0
               prev-word = 'nih'           False : True   =     67.7 : 1.0
               prev-word = 'c'             False : True   =     65.0 : 1.0
               prev-word = 'h'             False : True   =     63.2 : 1.0


In [0]:
classifier.show_most_informative_features() #gniCorpus 수정 전을 train set으로 설정했을 경우

Most Informative Features
                   punct = '?'             False : True   =   3409.5 : 1.0
               prev-word = 'etc'           False : True   =    924.8 : 1.0
               prev-word = 'condition'     False : True   =     64.0 : 1.0
               prev-word = 'rhythms'       False : True   =     64.0 : 1.0
               prev-word = 'angiosperms'   False : True   =     64.0 : 1.0
               prev-word = 'medicine'      False : True   =     49.8 : 1.0
               prev-word = 'authorization'  False : True   =     45.7 : 1.0
               prev-word = 'al'             True : False  =     44.0 : 1.0
               prev-word = 'plants'        False : True   =     38.4 : 1.0
               prev-word = 'i'             False : True   =     38.4 : 1.0


In [0]:
classifier.show_most_informative_features() #treebank를 train set으로 설정했을 경우

Most Informative Features
               prev-word = 'mr'            False : True   =    112.7 : 1.0
               prev-word = '3'             False : True   =     46.9 : 1.0
               prev-word = '7'             False : True   =     42.3 : 1.0
               prev-word = '2'             False : True   =     27.6 : 1.0
               prev-word = '6'             False : True   =     25.0 : 1.0
               prev-word = 'n'             False : True   =     19.4 : 1.0
   prev-word-is-one-char = True            False : True   =     15.3 : 1.0
               prev-word = '1'             False : True   =     15.2 : 1.0
               prev-word = 's'             False : True   =     14.9 : 1.0
               prev-word = 'corp'          False : True   =     14.7 : 1.0
